In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [19]:
import pandas as pd
import numpy as np
import time

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

In [48]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# 1. Загрузка данных
def load_data(filepath):
    """Загрузка данных из CSV файла"""
    df = pd.read_csv(filepath, sep=';')
    return df

# 2. Предобработка данных
def preprocess_data(df):
    """Предварительная обработка данных"""
    # Разделение на категориальные и некатегориальные признаки
    df_no_cat = df.drop(columns=['student_id', 'student_email', 'group_id', 'group_title', 
                               'prep_email', 'date_first_l', 'feat_group_format', 
                               'feat_payment_type', 'feat_payment_place'])
    df_cat = df[['feat_group_format', 'feat_payment_type', 'feat_payment_place']].copy()
    
    # Feature engineering
    # Добавление признака класса
    def init_class(group_code):
        for N_class in ['11', '10', '9', '8']:
            if N_class in str(group_code)[:15]:
                return int(N_class)
        return None
    
    df['feat_class'] = df['group_title'].apply(init_class)
    
    # Добавление признака формата группы
    def init_group_format(group_code):
        code_to_format = {
            'mini': 'Мини', 'bl': 'Блендед', 'class': 'Блендед',
            'onl+': 'Стандарт', 'micro': 'Микро', 'onl': 'Мини',
            'maximum': 'Микро', 'k+o': 'Онлайн+Класс', 'o+k': 'Онлайн+Класс'
        }
        group_code = str(group_code)
        for format_code, format_title in code_to_format.items():
            if format_code in group_code:
                return format_title
        return None
    
    df['feat_group_format'] = df['group_title'].apply(init_group_format)
    
    # Добавление признака длительности группы в неделях
    def init_group_length(group_code):
        group_code = str(group_code)
        pos = group_code.rfind('w_', 0, -10)
        if pos == -1:
            return None
        group_length = group_code[pos-2:pos].replace('_', '')
        return int(group_length) if group_length.isdigit() else None
    
    df['feat_group_weeks'] = df['group_title'].apply(init_group_length)
    
    # Обработка пропущенных значений
    df.drop(['feat_TK_right', 'feat_prep_SIM'], axis=1, inplace=True, errors='ignore')
    df['feat_payment_place'] = df['feat_payment_place'].fillna('ИК')
    df['feat_term_perc'] = df['feat_term_perc'].fillna(df['feat_term_perc'].mean())
    df['feat_prep_OpenLessons'] = df['feat_prep_OpenLessons'].fillna(df['feat_prep_OpenLessons'].mean())
    df['feat_prep_HW'] = df['feat_prep_HW'].fillna(df['feat_prep_HW'].mean())
    df['feat_prep_attendence'] = df['feat_prep_attendence'].fillna(df['feat_prep_attendence'].mean())
    df.dropna(subset=['prep_email'], inplace=True)
    
    return df_no_cat, df_cat, df

# 3. Кодирование категориальных признаков
def encode_categorical(df_cat):
    """One-Hot Encoding категориальных признаков"""
    ohe = OneHotEncoder(sparse_output=False, dtype=int)
    
    # Кодируем каждый категориальный признак отдельно
    encoded_dfs = []
    for col in df_cat.columns:
        encoded_df = pd.DataFrame(
            ohe.fit_transform(df_cat[[col]]),
            columns=ohe.get_feature_names_out([col])
        )
        encoded_dfs.append(encoded_df)
    
    # Объединяем все закодированные признаки
    encoded_ohe_df = pd.concat(encoded_dfs, axis=1)
    return encoded_ohe_df

# 4. Отбор признаков
def select_features(encoded_ohe_df, target_col='target', threshold=0.05):
    """Отбор признаков на основе корреляции с целевой переменной"""
    corr_matrix = encoded_ohe_df.corr()
    target_corr = corr_matrix[target_col].abs().sort_values(ascending=False)
    strong_features = target_corr[target_corr > threshold].index.tolist()
    print(f"Лучшие признаки: {strong_features}")
    return strong_features

# 5. Подготовка данных для моделирования
def prepare_data(encoded_ohe_df, strong_features, target_col='target'):
    """Подготовка данных для обучения модели"""
    df_few = encoded_ohe_df[strong_features]
    X = df_few.drop(target_col, axis=1)
    y = df_few[target_col]
    return X, y

# 6. Обработка дисбаланса классов
def balance_classes(X, y):
    """Балансировка классов с помощью RandomUnderSampler"""
    rus = RandomUnderSampler(random_state=42)
    X_under, y_under = rus.fit_resample(X, y)
    return train_test_split(X_under, y_under, test_size=0.2, random_state=42)

# 7. Обучение и оценка модели
def train_and_evaluate(X_train, X_test, y_train, y_test):
    """Обучение и оценка модели CatBoost"""
    # Обучение модели
    start = time.time()
    model = CatBoostClassifier(
        iterations=290,
        depth=9,
        random_state=42,
        learning_rate=0.04,
        verbose=0,
        task_type='GPU',
        boosting_type='Ordered'
    )
    model.fit(X_train, y_train)
    print(f'Время обучения модели: {round(time.time() - start,2)} секунд')
    
    # Оценка модели
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    # Кросс-валидация
    start = time.time()
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = cross_val_score(model, pd.concat([X_train, X_test]), pd.concat([y_train, y_test]), 
                             cv=skf, scoring='f1').mean()
    print(f'f-1 score: {round(cv_score,4)} \nВремя кроссвалидации: {round(time.time() - start,2)} секунд')
    
    return model

# Основной пайплайн
def main():
    # 1. Загрузка данных
    df = load_data('Clean_Data.csv')
    
    # 2. Предобработка данных
    df_no_cat, df_cat, df = preprocess_data(df)
    
    # 3. Кодирование категориальных признаков
    encoded_ohe_df = encode_categorical(df_cat)
    
    # Объединяем все признаки
    final_df = pd.concat([df_no_cat, encoded_ohe_df], axis=1)
    
    # 4. Отбор признаков
    strong_features = select_features(final_df)
    
    # 5. Подготовка данных для моделирования
    X, y = prepare_data(final_df, strong_features)
    
    # 6. Обработка дисбаланса классов
    X_train, X_test, y_train, y_test = balance_classes(X, y)
    
    # 7. Обучение и оценка модели
    model = train_and_evaluate(X_train, X_test, y_train, y_test)

if __name__ == "__main__":
    main()

Лучшие признаки: ['target', 'feat_good_agr_before', 'feat_attendance', 'feat_LB', 'feat_HW', 'feat_HW_right', 'feat_theory_read', 'feat_TK', 'feat_payment_type_Позтапная', 'feat_payment_type_Единовременная', 'feat_tickets_mb_term', 'feat_term_perc', 'feat_payment_place_Навигатор', 'feat_prep_HW']
Время обучения модели: 22.43 секунд
              precision    recall  f1-score   support

           0       0.81      0.88      0.84      1801
           1       0.87      0.80      0.83      1817

    accuracy                           0.84      3618
   macro avg       0.84      0.84      0.84      3618
weighted avg       0.84      0.84      0.84      3618

f-1 score: 0.8374 
Время кроссвалидации: 112.51 секунд


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
import time

# 1. Кастомный трансформер для feature engineering
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        
        # Извлечение класса из group_title
        def init_class(group_code):
            for N_class in ['11', '10', '9', '8']:
                if N_class in str(group_code)[:15]:
                    return int(N_class)
            return np.nan
        
        df['feat_class'] = df['group_title'].apply(init_class)
        
        # Извлечение формата группы
        def init_group_format(group_code):
            code_to_format = {
                'mini': 'Мини', 'bl': 'Блендед', 'class': 'Блендед',
                'onl+': 'Стандарт', 'micro': 'Микро', 'onl': 'Мини',
                'maximum': 'Микро', 'k+o': 'Онлайн+Класс', 'o+k': 'Онлайн+Класс'
            }
            group_code = str(group_code)
            for format_code, format_title in code_to_format.items():
                if format_code in group_code:
                    return format_title
            return np.nan
        
        df['feat_group_format'] = df['group_title'].apply(init_group_format)
        
        # Извлечение длительности группы в неделях
        def init_group_length(group_code):
            group_code = str(group_code)
            pos = group_code.rfind('w_', 0, -10)
            if pos == -1:
                return np.nan
            group_length = group_code[pos-2:pos].replace('_', '')
            return int(group_length) if group_length.isdigit() else np.nan
        
        df['feat_group_weeks'] = df['group_title'].apply(init_group_length)
        
        # Обработка пропущенных значений
        df['feat_payment_place'] = df['feat_payment_place'].fillna('ИК')
        num_cols = ['feat_term_perc', 'feat_prep_OpenLessons', 'feat_prep_HW', 'feat_prep_attendence']
        for col in num_cols:
            df[col] = df[col].fillna(df[col].mean())
        
        # Удаление ненужных колонок
        cols_to_drop = ['student_id', 'student_email', 'group_id', 'group_title', 
                       'prep_email', 'date_first_l', 'feat_TK_right', 'feat_prep_SIM']
        df.drop([col for col in cols_to_drop if col in df.columns], axis=1, inplace=True)
        
        return df

# 2. Загрузка данных
def load_data(filepath):
    """Загрузка данных из CSV файла"""
    df = pd.read_csv(filepath, sep=';')
    return df

# 3. Основной пайплайн
def main():
    # Загрузка данных
    df = load_data('Clean_Data.csv')
    
    # Разделение на признаки и целевую переменную
    X = df.drop('target', axis=1)
    y = df['target']
    
    # Определение категориальных и числовых признаков
    categorical_features = ['feat_group_format', 'feat_payment_type', 'feat_payment_place']
    numeric_features = [col for col in X.columns 
                       if col not in categorical_features + ['student_id', 'student_email', 'group_id', 
                                                          'group_title', 'prep_email', 'date_first_l']]
    
    # Создание пайплайна
    pipeline = ImbPipeline([
        # Шаг 1: Feature Engineering
        ('feature_engineering', FeatureEngineer()),
        
        # Шаг 2: Препроцессинг
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('num', 'passthrough', numeric_features),
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
            ])),
        
        # Шаг 3: Устранение дисбаланса классов
        ('undersampler', RandomUnderSampler(random_state=42)),
        
        # Шаг 4: Модель
        ('classifier', CatBoostClassifier(
            iterations=290,
            depth=9,
            random_state=42,
            learning_rate=0.04,
            verbose=0,
            task_type='GPU',
            boosting_type='Ordered'
        ))
    ])
    
    # Разделение данных на train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Обучение пайплайна
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    print(f"Время обучения: {time.time() - start_time:.2f} секунд")
    
    # Предсказание на тестовых данных
    
    y_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    # Кросс-валидация
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X, y, cv=skf, scoring='f1')
    print(f"F1-score (кросс-валидация): {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

if __name__ == "__main__":
    main()

Время обучения: 26.76 секунд
              precision    recall  f1-score   support

           0       0.97      0.88      0.92     11659
           1       0.52      0.82      0.64      1809

    accuracy                           0.87     13468
   macro avg       0.75      0.85      0.78     13468
weighted avg       0.91      0.87      0.89     13468

